In [5]:
from scapy.all import *

## Filter out RST Packets

In [6]:
!iptables --help

iptables v1.8.7

Usage: iptables -[ACD] chain rule-specification [options]
	iptables -I chain [rulenum] rule-specification [options]
	iptables -R chain rulenum rule-specification [options]
	iptables -D chain rulenum [options]
	iptables -[LS] [chain [rulenum]] [options]
	iptables -[FZ] [chain] [options]
	iptables -[NX] chain
	iptables -E old-chain-name new-chain-name
	iptables -P chain target [options]
	iptables -h (print this help information)

Commands:
Either long or short options are allowed.
  --append  -A chain		Append to chain
  --check   -C chain		Check for the existence of a rule
  --delete  -D chain		Delete matching rule from chain
  --delete  -D chain rulenum
				Delete rule rulenum (1 = first) from chain
  --insert  -I chain [rulenum]
				Insert in chain as rulenum (default 1=first)
  --replace -R chain rulenum
				Replace rule rulenum (1 = first) in chain
  --list    -L [chain [rulenum]]
				List the rules in a chain or all chains
  --list-rules -S [chain [rulenum]]
				Pri

##### Outbound 
```bash
$ iptables -A OUTPUT -p tcp --tcp-flags RST RST -j DROP
```


In [7]:
!ip a

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: bond0: <BROADCAST,MULTICAST,MASTER> mtu 1500 qdisc noop state DOWN group default qlen 1000
    link/ether 12:a6:e9:68:d8:2b brd ff:ff:ff:ff:ff:ff
3: dummy0: <BROADCAST,NOARP> mtu 1500 qdisc noop state DOWN group default qlen 1000
    link/ether 16:40:2c:73:15:73 brd ff:ff:ff:ff:ff:ff
4: tunl0@NONE: <NOARP> mtu 1480 qdisc noop state DOWN group default qlen 1000
    link/ipip 0.0.0.0 brd 0.0.0.0
5: sit0@NONE: <NOARP> mtu 1480 qdisc noop state DOWN group default qlen 1000
    link/sit 0.0.0.0 brd 0.0.0.0
6: eth0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 00:15:5d:0b:8e:85 brd ff:ff:ff:ff:ff:ff
    inet 172.17.92.177/20 brd 17

#### TCP RST Attack

In [104]:
ip_host= "192.168.1.3"

In [86]:

# SYN
ip=IP(dst=ip_host)
SYN=TCP(sport=22,dport=22,flags='S',seq=1000)
SYNACK=sr1(ip/SYN,timeout=5)



Begin emission:
Finished sending 1 packets.
*
Received 1 packets, got 1 answers, remaining 0 packets


In [103]:
SYNACK

<IP  version=4 ihl=5 tos=0x0 len=44 id=0 flags=DF frag=0 ttl=63 proto=tcp chksum=0x715e src=192.168.1.3 dst=172.17.92.177 |<TCP  sport=ssh dport=ssh seq=1966037757 ack=1001 dataofs=6 reserved=0 flags=SA window=64240 chksum=0xfe75 urgptr=0 options=[('MSS', 1460)] |>>

In [87]:
SYNACK.ack

1001

In [88]:
SYNACK.seq

1966037757

In [89]:
# SYN-ACK
ACK=TCP(sport=sport, dport=22, flags='A', seq=SYNACK.ack + 1, ack=SYNACK.seq + 1)
send(ip/ACK)


Sent 1 packets..


#### Host respond
![alt text](tcp_rst_ack.png "tcp_rst_ack")

#### Now block TCP RST 

```bash
#Inbound

$ iptables -I INPUT -p tcp --tcp-flags ALL RST,ACK -j DROP
$ iptables -I INPUT -p tcp --tcp-flags ALL RST -j DROP

#Outbound

$ iptables -I OUTPUT -p tcp --tcp-flags ALL RST,ACK -j DROP
```

#### Current Rules:
```bash
(base) ┌─[uninorte@uninorte-dell] - [/code/nets] - [jue jul 28, 15:38]
└─[$] <> sudo iptables -L OUTPUT --line-numbers
Chain OUTPUT (policy ACCEPT)
num  target     prot opt source               destination
1    DROP       tcp  --  anywhere             anywhere             tcp flags:FIN,SYN,RST,PSH,ACK,URG/RST,ACK
(base) ┌─[uninorte@uninorte-dell] - [/code/nets] - [jue jul 28, 15:38]
└─[$] <> sudo iptables -L INPUT --line-numbers
Chain INPUT (policy ACCEPT)
num  target     prot opt source               destination
1    DROP       tcp  --  anywhere             anywhere             tcp flags:FIN,SYN,RST,PSH,ACK,URG/RST
2    DROP       tcp  --  anywhere             anywhere             tcp flags:FIN,SYN,RST,PSH,ACK,URG/RST,ACK
(base) ┌─[uninorte@uninorte-dell] - [/code/nets] - [jue jul 28, 15:39]
```

In [98]:
# SYN
ip2=IP(dst=ip_host)
SYN2=TCP(sport=1500,dport=5000,flags='S',seq=1000)
SYNACK2=sr1(ip2/SYN2,timeout=5)


Begin emission:
Finished sending 1 packets.

Received 0 packets, got 0 answers, remaining 1 packets


In [102]:
print(SYNACK2)

None


## TraceRoute

In [148]:
## TCP traceroute

target = ["192.168.1.1"]
result, unans = traceroute(target,maxttl=4)

Begin emission:
Finished sending 4 packets.
*
Received 1 packets, got 1 answers, remaining 3 packets
  192.168.1.1:tcp80  
1 172.17.80.1     11 


In [177]:
target = ["waybarrios.com"]
result, unans = traceroute(target,maxttl=8)

Begin emission:
Finished sending 8 packets.
****
Received 4 packets, got 4 answers, remaining 4 packets
  104.21.66.126:tcp80 
1 172.17.80.1     11  
2 192.168.1.1     11  
3 186.102.160.2   11  
6 192.168.100.53  11  


In [ ]:
#UDP
def traceroute_udp(host,maxttl=4,dport=33434): 
    for i in range(1, maxttl):
        pkt = IP(dst=host, ttl=i) / UDP(dport=dport)
        # Send the packet and get a reply
        reply = sr1(pkt, verbose=0,timeout=2)
        if reply is None:
            break
        elif reply.type == 3:
            # We've reached our destination
            print ("Done!", reply.src)
            break
        else:
            # We're in the middle somewhere
            print ("%d Iteration: " % i , reply.src)

In [252]:
traceroute_udp("192.168.1.1")

1 Iteration:  172.17.80.1
Done! 192.168.1.1


In [213]:
traceroute_udp("waybarrios.com",maxttl=10)

1 Iteration:  172.17.80.1
2 Iteration:  192.168.1.1
3 Iteration:  186.102.160.2
4 Iteration:  206.223.124.156
5 Iteration:  206.223.124.154
6 Iteration:  192.168.100.53


In [255]:
def traceroute_icmp(host):
    print("Tracroute", host)
    flag = True
    ttl=1
    hops = []
    while flag:
        ans, unans = sr(IP(dst=host,ttl=ttl)/ICMP(),verbose=0)
        if ans.res[0][1].type == 0: # checking for  ICMP echo-reply
                hops.append(ans.res[0][1].src)
                flag = False
        else:
                hops.append(ans.res[0][1].src) # storing the src ip from ICMP error message
                ttl +=1

    for i,hop in enumerate(hops):
        print (i, " " + hop)
    return hops

In [256]:
_ = traceroute_icmp("8.8.8.8")

Tracroute 8.8.8.8
0  172.17.80.1
1  192.168.1.1
2  186.102.160.2
3  72.14.222.115
4  72.14.222.114
5  108.170.253.209
6  142.250.231.163
7  8.8.8.8


In [257]:
res, unans = traceroute(["www.microsoft.com",],dport=[80,443],maxttl=20,retry=-2)

Begin emission:
Finished sending 40 packets.
************************************Begin emission:
Finished sending 4 packets.
Begin emission:
Finished sending 4 packets.

Received 36 packets, got 36 answers, remaining 4 packets
   23.14.36.240:tcp443 23.14.36.240:tcp80 
1  172.17.80.1     11  172.17.80.1     11 
2  192.168.1.1     11  192.168.1.1     11 
3  186.102.160.2   11  186.102.160.2   11 
6  94.142.99.158   11  94.142.99.93    11 
7  23.14.36.240    SA  23.14.36.240    SA 
8  23.14.36.240    SA  23.14.36.240    SA 
9  23.14.36.240    SA  23.14.36.240    SA 
10 23.14.36.240    SA  23.14.36.240    SA 
11 23.14.36.240    SA  23.14.36.240    SA 
12 23.14.36.240    SA  23.14.36.240    SA 
13 23.14.36.240    SA  23.14.36.240    SA 
14 23.14.36.240    SA  23.14.36.240    SA 
15 23.14.36.240    SA  23.14.36.240    SA 
16 23.14.36.240    SA  23.14.36.240    SA 
17 23.14.36.240    SA  23.14.36.240    SA 
18 23.14.36.240    SA  23.14.36.240    SA 
19 23.14.36.240    SA  23.14.36.240    SA 